In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

## API 概览

本笔记演示了如何使用 API（而非 CLI）与 graphrag 库进行交互。请注意，graphrag 的 CLI 实际上通过此 API 连接到库并执行所有操作。

In [ ]:
from pathlib import Path
from pprint import pprint

import pandas as pd

import graphrag.api as api
from graphrag.config.load_config import load_config
from graphrag.index.typing.pipeline_run_result import PipelineRunResult

In [ ]:
PROJECT_DIRECTORY = "<your project directory>"

## 先决条件
所有 API 操作都需要一个 `GraphRagConfig` 对象。它是控制 graphrag 行为的主要手段，可以通过 `settings.yaml` 配置文件实例化。

有关如何生成 `settings.yaml` 文件的更多详细信息，请参阅 [CLI 文档](https://microsoft.github.io/graphrag/cli/#init)。

### Generate a `GraphRagConfig` object

In [ ]:
graphrag_config = load_config(Path(PROJECT_DIRECTORY))

## 索引 API

*索引*是提取原始文本数据并构建知识图谱的过程。GraphRAG 目前支持纯文本 (.txt) 和 .csv 文件格式。

## 建立索引

In [ ]:
index_result: list[PipelineRunResult] = await api.build_index(config=graphrag_config)

# index_result is a list of workflows that make up the indexing pipeline that was run
for workflow_result in index_result:
    status = f"error\n{workflow_result.errors}" if workflow_result.errors else "success"
    print(f"Workflow Name: {workflow_result.workflow}\tStatus: {status}")

## 查询索引

要查询索引，首先必须将多个索引文件读入内存并传递给查询 API。

In [ ]:
entities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/entities.parquet")
communities = pd.read_parquet(f"{PROJECT_DIRECTORY}/output/communities.parquet")
community_reports = pd.read_parquet(
    f"{PROJECT_DIRECTORY}/output/community_reports.parquet"
)

response, context = await api.global_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query="Who is Scrooge and what are his main relationships?",
)

响应对象是来自 graphrag 的官方响应，而上下文对象包含有关用于获取最终响应的查询过程的各种元数据。

In [ ]:
print(response)

进一步挖掘上下文可以为用户提供极其细化的信息，例如最终检索到哪些数据源（细化到文本块级别）并将其用作发送到 LLM 模型的上下文的一部分。

In [ ]:
pprint(context)  # noqa: T203